In [57]:
# imports
import yaml
import json
import numpy as np
from groups import population
from strategy import Strategy
from testing_regime import TestingRegime
from transform import transform
from sim_helper import sim_test_strategy
import plotting

In [52]:
# get parameters
nominal_scenario = yaml.safe_load(open("nominal.yaml", "r"))
json_params = json.load(open(nominal_scenario["json_path"]))
nominal_scenario.update(json_params)

nominal_scenario["meta_matrix"] = \
    np.array([list(row.values()) for row in nominal_scenario["meta_matrix"].values()])

In [53]:
T = nominal_scenario['T']
CLASSWORK_TRANSMISSION_MULTIPLIER = \
    list(nominal_scenario['classwork_transmission_multiplier'].values())

In [54]:
popul = population.from_scenario(nominal_scenario)

no_testing_testing_regime=TestingRegime(popul=popul, tests_per_week=0,
                                        test_delay=1, params=nominal_scenario)

ug_prof_2x_week_testing_regime= \
    TestingRegime(popul=popul, tests_per_week={ 'UG':2, 'GR':0, 'PR':2, 'FS':0},
                  test_delay=1, params=nominal_scenario)

surge_testing_strategy = \
    Strategy(name="UG+Prof. 2x/wk in Virtual Instr. Only",
         # pct_discovered_in_pre_departure=0.5,   # Using Scenario 1
         # pct_discovered_in_arrival_test=0.335,  # Using Scenario 1
         pct_discovered_in_pre_departure=0.25,  # Using Scenario 2
         pct_discovered_in_arrival_test=0.38,  # Using Scenario 2
         testing_regimes=[ug_prof_2x_week_testing_regime,
                          ug_prof_2x_week_testing_regime,
                          no_testing_testing_regime],
         transmission_multipliers=[1, CLASSWORK_TRANSMISSION_MULTIPLIER,
                                   CLASSWORK_TRANSMISSION_MULTIPLIER],
         period_lengths=[3,3,T-6-1])

In [55]:
trajectories = []
for i in np.linspace(0,1,10):
    scenario = transform(nominal_scenario, {"winter_break_infections_global": i})
    trajectories.append(sim_test_strategy(scenario, popul, surge_testing_strategy, 'black'))

In [59]:
trajectories[0]